# Homework #3 #

## Due 11:59 pm Feb 27  (Submit all work including computer codes) ###

**1. (60 points) Program Polak-Ribiere and Fletcher-Reeves conjugate gradient methods with exact line search to minimize the Rosenbrock function:**
$ f(x) = 100 ( x^2 − x_{1}^2 )^2 + ( 1 − x_{1} )^2 $

### Part A ###

Choose initial point $(1.2, 1.2)$, set tolerance (say $1 * e^{-8}$, adjust if necessary) on the norm of gradient, and study convergence (linear or super-linear) for each method. How do they compare ?

In [1]:
import numpy as np

In [2]:
f = lambda x,y: 100*(y-x**2)**2 + (1-x)**2
Df = lambda x,y: np.array([(200*(y-x**2)*(-2)*x + 2*(-1)*(y-x**2)),(200*(y-x**2))])

x0,y0 = 1.2,1.2    # initial point for all methods

In [3]:
def WolfeI(alpha,f,x,dx,p,c1=0.1):
    '''Return True/False if Wolfe condition I is satisfied for the given alpha'''
    LHS = f(x[0]+alpha*p[0], x[1]+alpha*p[1])
    RHS = f(x[0],x[1])+c1*alpha*np.dot(dx,p)
    return LHS <= RHS

In [4]:
def PR_beta(Dfk,Dfk1, pk): #doesn't use pk, just including the parameter since other beta methods use pk
    return ((Dfk1-Dfk)@Dfk1) / (Dfk@Dfk)

def FR_beta(Dfk,Dfk1, pk): #doesn't use pk
    return (Dfk1@Dfk1) / (Dfk@Dfk)

In [5]:
def nonlinearCG(x0, y0, f, Df, beta, restart=True, tol=1e-8, max_steps=10000, rho=0.75):
    x,y = x0,y0
    path = [[x,y]]
    i=0                   # iteration count
    dx = Df(x,y)          # current gradient
    pk = -dx
    while np.linalg.norm(dx)>tol and i<max_steps:    
        # backtracking
        alpha = 1
        while not WolfeI(alpha,f,np.array([x,y]),dx,pk) and alpha>1e-5: # lower bound on alpha attempts to force Wolfe II
            alpha *= rho

        xnew,ynew = x + alpha*pk[0], y + alpha*pk[1]
        dx1 = Df(xnew,ynew)      # Df_{k+1}
        bk = beta(dx,dx1,pk)  # beta_k
        if restart and i%2 == 0:
            bk=0
        pk = -dx1 + bk*pk
        path.append([xnew,ynew])
        x,y = xnew,ynew
        dx = dx1
        i += 1
    return np.array(path), i, x, y

In [6]:
path_PR, i_PR, x_PR, y_PR = nonlinearCG(x0, y0, f, Df, PR_beta, True)
print(f'After {i_PR} iterations, approximate minimum for PR is {f(x_PR,y_PR)} at {x_PR,y_PR}')
path_FR, i_FR, x_FR, y_FR = nonlinearCG(x0, y0, f, Df, FR_beta, True)
print(f'After {i_FR} iterations, approximate minimum for FR is {f(x_FR,y_FR)} at {x_FR,y_FR}')

After 1853 iterations, approximate minimum for PR is 0.012043849958000773 at (1.109744475751633, 1.2315328014816844)
After 1163 iterations, approximate minimum for FR is 0.011759300258886746 at (1.1084403073533395, 1.2286399149856848)


The Polak-Ribiere and Fletcher-Reeves conjugate gradient methods yield a similar result, although the Flecher-Reeves conjugate gradient method took less iterations to get to that result.

### Part B ###

Repeat A by performing direction vector restart every 3 iterations. 

In [7]:
def nonlinearCG(x0, y0, f, Df, beta, restart=True, tol=1e-8, max_steps=10000, rho=0.75):
    x,y = x0,y0
    path = [[x,y]]
    i=0                   # iteration count
    dx = Df(x,y)          # current gradient
    pk = -dx
    while np.linalg.norm(dx)>tol and i<max_steps:    
        # backtracking
        alpha = 1
        while not WolfeI(alpha,f,np.array([x,y]),dx,pk) and alpha>1e-5: # lower bound on alpha attempts to force Wolfe II
            alpha *= rho

        xnew,ynew = x + alpha*pk[0], y + alpha*pk[1]
        dx1 = Df(xnew,ynew)      # Df_{k+1}
        bk = beta(dx,dx1,pk)  # beta_k
        if restart and i%3 == 0:
            bk=0
        pk = -dx1 + bk*pk
        path.append([xnew,ynew])
        x,y = xnew,ynew
        dx = dx1
        i += 1
    return np.array(path), i, x, y

In [8]:
f = lambda x,y: 100*(y-x**2)**2 + (1-x)**2
Df = lambda x,y: np.array([(200*(y-x**2)*(-2)*x + 2*(-1)*(y-x**2)),(200*(y-x**2))])
x0,y0 = 1.2,1.2    # initial point for all methods

path_PR, i_PR, x_PR, y_PR = nonlinearCG(x0, y0, f, Df, PR_beta, True)
print(f'After {i_PR} iterations, approximate minimum for PR with Restart is {f(x_PR,y_PR)} at {x_PR,y_PR}')


path_PRf, i_PRf, x_PRf, y_PRf = nonlinearCG(x0, y0, f, Df, PR_beta, False)
print(f'After {i_PRf} iterations, approximate minimum for PR without Restart is {f(x_PRf,y_PRf)} at {x_PRf,y_PRf}')

path_FR, i_FR, x_FR, y_FR = nonlinearCG(x0, y0, f, Df, FR_beta, True)
print(f'After {i_FR} iterations, approximate minimum for FR with Restart is {f(x_FR,y_FR)} at {x_FR,y_FR}')

path_FRf, i_FRf, x_FRf, y_FRf = nonlinearCG(x0, y0, f, Df, FR_beta, False)
print(f'After {i_FRf} iterations, approximate minimum for FR without Restart is {f(x_FRf,y_FRf)} at {x_FRf,y_FRf}')

After 1843 iterations, approximate minimum for PR with Restart is 0.012043894337510165 at (1.1097446779461773, 1.2315332502502776)
After 17 iterations, approximate minimum for PR without Restart is nan at (nan, inf)


<ipython-input-8-5c1ff550b486>:1: RuntimeWarning: overflow encountered in double_scalars
  f = lambda x,y: 100*(y-x**2)**2 + (1-x)**2
<ipython-input-8-5c1ff550b486>:2: RuntimeWarning: overflow encountered in double_scalars
  Df = lambda x,y: np.array([(200*(y-x**2)*(-2)*x + 2*(-1)*(y-x**2)),(200*(y-x**2))])
<ipython-input-7-265df3550712>:18: RuntimeWarning: invalid value encountered in add
  pk = -dx1 + bk*pk


After 880 iterations, approximate minimum for FR with Restart is 0.01160854303448537 at (1.1077429488852304, 1.2270944408251279)
After 185 iterations, approximate minimum for FR without Restart is 0.010182359478602786 at (1.1009076779962892, 1.2119977154904495)


### Part C ###

Repeat A & B for initial point $(-1.2, 1)$.

In [9]:
f = lambda x,y: 100*(y-x**2)**2 + (1-x)**2
Df = lambda x,y: np.array([(200*(y-x**2)*(-2)*x + 2*(-1)*(y-x**2)),(200*(y-x**2))])
x0,y0 = -1.2,1    # initial point for all methods

path_PR, i_PR, x_PR, y_PR = nonlinearCG(x0, y0, f, Df, PR_beta, True)
print(f'After {i_PR} iterations, approximate minimum for PR with Restart is {f(x_PR,y_PR)} at {x_PR,y_PR}')


path_PRf, i_PRf, x_PRf, y_PRf = nonlinearCG(x0, y0, f, Df, PR_beta, False)
print(f'After {i_PRf} iterations, approximate minimum for PR without Restart is {f(x_PRf,y_PRf)} at {x_PRf,y_PRf}')

path_FR, i_FR, x_FR, y_FR = nonlinearCG(x0, y0, f, Df, FR_beta, True)
print(f'After {i_FR} iterations, approximate minimum for FR with Restart is {f(x_FR,y_FR)} at {x_FR,y_FR}')

path_FRf, i_FRf, x_FRf, y_FRf = nonlinearCG(x0, y0, f, Df, FR_beta, False)
print(f'After {i_FRf} iterations, approximate minimum for FR without Restart is {f(x_FRf,y_FRf)} at {x_FRf,y_FRf}')

After 2520 iterations, approximate minimum for PR with Restart is 4.100734660868428 at (-1.0250270765766143, 1.0506805077371888)
After 2479 iterations, approximate minimum for PR without Restart is 4.100238583397993 at (-1.0249045862454835, 1.0504294109289871)
After 1087 iterations, approximate minimum for FR with Restart is 4.012776047287621 at (-1.0031914654589613, 1.00639311639205)
After 261 iterations, approximate minimum for FR without Restart is 3.1634157456502616 at (-0.7785993775019324, 0.6062169906720151)
